###### 작성자 : 손해원


#### 4월 6일 최종수정 

In [1]:
from selenium import webdriver
import pandas as pd
import numpy as np
from tqdm import tqdm
import time
from glob import glob
from bs4 import BeautifulSoup
from urllib.request import Request, urlopen
from tqdm import tqdm
# 멀티스레딩
from multiprocessing.pool import ThreadPool

In [2]:
# 각 업종별 기업 크롤링 함수 

def crawl_(address,name):

    ###########################  크롬드라이버 경로를 확인해 주세요 ! 
    driver = webdriver.Chrome('/Users/user/Jupyter/driver/chromedriver')
    # 네이버 증권홈 -> 국내증시 -> 업종별시세: url에 업종별 number 지정 
    # 각 업종 페이지로 이동 
    driver.get(address)

    # 기본 선택됨 : 거래량, 매수호가, 거래대금, 매도호가, 전일거래량
    # 모두 선택 해제 후 다시 선택 -> 시가총액, per    
    georae_yang = driver.find_element_by_id('option1') # 거래량
    maesoo_hoga = driver.find_element_by_id('option2') # 매수호가
    georae_daegum = driver.find_element_by_id('option3') # 거래대금
    maedo_hoga = driver.find_element_by_id('option8') # 매도호가
    jeon_il_geora = driver.find_element_by_id('option9') # 전일거래량 

    # 선택해제 
    georae_yang.click()
    maesoo_hoga.click()
    georae_daegum.click()
    maedo_hoga.click()
    jeon_il_geora.click()
    
    # 시가총액, per 선택 
    siga_chong = driver.find_element_by_id('option4') # 시가총액
    per = driver.find_element_by_id('option6') # per

    # 선택 
    siga_chong.click()
    per.click()

    # 적용하기 클릭 
    xpath = '''//*[@id="contentarea"]/div[3]/form/div/div/div/a[1]'''
    apply = driver.find_element_by_xpath(xpath)
    apply.click()

    # 기업명, 시가총액, per 가져오기 
    name_list = []
    market_cap_list = []
    per_list = []
    
    # 테이블 
    xpath = '''//*[@id="contentarea"]/div[4]/table/tbody'''
    table = driver.find_element_by_xpath(xpath).text

    # 테이블 안의 이름 영역 
    name_area = driver.find_elements_by_class_name('name_area')
    # 테이블 안의 모든 숫자 -> td class="number" 
    number_area = driver.find_elements_by_class_name('number')

    # 기업명
    name_list = [x.find_element_by_tag_name('a').text for x in name_area]

    # 시가총액, per
    for i in range(len(number_area)):
        # 3번째 인덱스인 시가총액
        if(i%5==3):
            market_cap_list.append((number_area[i].text).replace(',',''))
        # 4번째 인덱스인 per
        elif(i%5==4):
            per_list.append(number_area[i].text)

    # 첫 번째 값이 이상하당 빼버리자 
    market_cap_list = market_cap_list[1:]
    per_list = per_list[1:]

    # 기업 페이지 url
    name_box = driver.find_elements_by_class_name('name_area')
    company_page = [x.find_element_by_tag_name('a').get_attribute('href')  for x in name_box]

    # 각 기업 페이지 접속 -> 종목코드/클래스/상장일 저장
    # 84번 반복합니다 ..! 

    code_list = []
    class_list = []
    reg_day_list = []

    for i in tqdm(range(len(company_page))):
        driver.get(company_page[i])

        # 종목코드 
        code = driver.find_element_by_class_name('code').text
        # print('종목코드는: ', code)
        code_list.append('A'+code)

        # 코스피,코스닥 클래스 
        try:
            driver.find_element_by_class_name('kospi')
            class_ = 'kospi'
        except:
            class_ = 'kosdaq'

        # print('코스피 or 코스닥? :', class_)
        class_list.append(class_)


        # 상장일 
        # url에 종목코드 넣어서 포맷 
        driver.get('https://navercomp.wisereport.co.kr/v2/company/c1020001.aspx?cmp_cd={code}&cn='.format(code=code))

        # 기업개요 클릭
        btn = driver.find_element_by_class_name('line-left')
        btn.click()

        reg_day_long = driver.find_element_by_css_selector('#cTB201 > tbody > tr:nth-child(3) > td.c2.txt').text
        x = reg_day_long.split()
        reg_day = x[-1].replace(')','')
        # print('상장일: ',reg_day)
        reg_day_list.append(reg_day)

    # 종료
    driver.close()


    # 데이터프레임화 합니다 
    df = pd.DataFrame(
        {
             '기업명':name_list
            ,'시가총액':market_cap_list
            ,'per':per_list
            ,'종목코드':code_list
            ,'클래스':class_list
            ,'상장일':reg_day_list
        }
    )
    # csv로 만들어줘 ~~~~~~
    # 여러번 실행하다보니 파일이름에 날짜를 추가하는 형식으로 설정했습니다. 
    pd.DataFrame(df).to_csv('./naver_crawling/{}_0406.csv'.format(name))


In [3]:
# 업종명 - url 리스트들을 4개로 나눕니당 

def quarter_splitter(lists):
    for company_info in lists:
        try:
            crawl_(company_info[0],company_info[1])
        except:
            continue

In [4]:
######## 실행코드
######## 환경에 따라 다르겠지만 필자는 30분정도 소요되었습니다 ...
######## 마음의 준비를 하신 후 실행해 주세요....!..! 

##########################  크롬드라이버 경로 확인 ! 
driver = webdriver.Chrome('/Users/user/Jupyter/driver/chromedriver')
# 네이버 증권홈 -> 국내증시 -> 업종별시세: url에 업종별 number 지정 
# 각 업종 페이지로 이동 
driver.get('https://finance.naver.com/sise/sise_group.nhn?type=upjong')

# 어떤 업종을 선택하시겠습니까 ,,! ,,! ,,! 
a_tags = driver.find_element_by_tag_name('tbody').find_elements_by_tag_name('a')
address = [x.get_attribute('href') for x in a_tags]
# 업종명은 
com_name = [x.text for x in a_tags]

# url,업종명 으로 묶인 리스트를 만든다 
addr_name_list=[]
for x,y in zip(address,com_name):
    addr_name_list.append([x,y])
# addr_name_list


# 전체 업종을 4등분해서, 4개의 cpu 프로세서 한개당 하나의 덩어리씩 맡아서 일해라! 라고 던져줍니당 
addr_name1=[]
addr_name2=[]
addr_name3=[]
addr_name4=[]

# 리스트 4등분하고, 한번씩 더 묶어주기 
for i in range(len(addr_name_list)):
    if i%4==0: addr_name1.append(addr_name_list[i])
    elif i%4==1: addr_name2.append(addr_name_list[i])
    elif i%4==2: addr_name3.append(addr_name_list[i])
    elif i%4==3: addr_name4.append(addr_name_list[i])
new_addr_name=[[addr_name1],[addr_name2],[addr_name3],[addr_name4]]

# 일해라 ~~~~~~ 
with ThreadPool(4) as tp:
    tp.starmap(quarter_splitter,new_addr_name)



  0%|          | 0/11 [00:00<?, ?it/s]

  0%|          | 0/3 [00:00<?, ?it/s]

  9%|▉         | 1/11 [00:09<01:31,  9.12s/it]


 25%|██▌       | 1/4 [00:18<00:54, 18.12s/it][A

 50%|█████     | 2/4 [00:21<00:27, 13.66s/it]]

100%|██████████| 3/3 [00:17<00:00,  5.90s/it]

100%|██████████| 4/4 [00:30<00:00,  7.55s/it]]

 45%|████▌     | 5/11 [00:31<00:36,  6.07s/it]


  1%|▏         | 1/70 [00:20<23:08, 20.12s/it]


 55%|█████▍    | 6/11 [00:58<01:02, 12.41s/it]

 64%|██████▎   | 7/11 [01:12<00:51, 12.92s/it]


 73%|███████▎  | 8/11 [01:20<00:33, 11.30s/it]


  6%|▌         | 4/70 [01:16<20:43, 18.85s/it]


  7%|▋         | 5/70 [01:17<14:41, 13.57s/it]


 82%|████████▏ | 9/11 [01:33<00:23, 11.72s/it]


 91%|█████████ | 10/11 [01:43<00:11, 11.29s/it]


 11%|█▏        | 8/70 [01:31<08:16,  8.02s/it]

  5%|▌         | 1/19 [00:59<17:54, 59.71s/it]


 13%|█▎        | 9/70 [01:46<10:24, 10.24s/it]


 14%|█▍        | 10/70 [01:52<08:51,  8.85s/it]

 10%|▉         | 2/21 [01:35<14:55, 47.13s/i

 52%|█████▏    | 23/44 [02:08<05:54, 16.88s/it][A


 89%|████████▊ | 62/70 [08:08<01:50, 13.81s/it]

 19%|█▉        | 4/21 [01:30<06:13, 21.95s/it]


 55%|█████▍    | 24/44 [02:27<05:54, 17.74s/it]


 24%|██▍       | 5/21 [01:44<05:13, 19.59s/it]


 93%|█████████▎| 65/70 [08:29<00:44,  8.99s/it]


 29%|██▊       | 6/21 [01:49<03:45, 15.04s/it]


 59%|█████▉    | 26/44 [02:46<03:49, 12.75s/it]


 33%|███▎      | 7/21 [02:09<03:53, 16.68s/it]


 99%|█████████▊| 69/70 [08:52<00:07,  7.98s/it]

 64%|██████▎   | 28/44 [03:28<04:44, 17.79s/it][A


100%|██████████| 70/70 [09:24<00:00,  8.06s/it]


 66%|██████▌   | 29/44 [03:51<04:50, 19.37s/it]

 48%|████▊     | 10/21 [03:14<03:23, 18.48s/it]

 52%|█████▏    | 11/21 [03:16<02:15, 13.56s/it]

 70%|███████   | 31/44 [04:19<03:37, 16.76s/it]

 45%|████▌     | 25/55 [04:12<06:41, 13.38s/it]

 62%|██████▏   | 13/21 [03:30<01:19,  9.89s/it]


 67%|██████▋   | 14/21 [03:32<00:52,  7.57s/it]

 71%|███████▏  | 15/21 [03:35<00:37,  6.29s/it]

 77%|████

 80%|████████  | 24/30 [02:03<00:23,  3.88s/it]

 87%|████████▋ | 13/15 [00:46<00:09,  4.50s/it]


 21%|██▏       | 17/80 [00:59<04:02,  3.86s/it]A


 62%|██████▎   | 5/8 [00:23<00:13,  4.35s/it]

 22%|██▎       | 18/80 [01:02<03:47,  3.66s/it]

100%|██████████| 15/15 [00:53<00:00,  3.54s/it]



 25%|██▌       | 20/80 [01:07<03:13,  3.22s/it]


 97%|█████████▋| 29/30 [02:17<00:02,  2.86s/it]


100%|██████████| 30/30 [02:20<00:00,  4.68s/it]

 29%|██▉       | 23/80 [01:17<03:11,  3.36s/it]

 31%|███▏      | 25/80 [01:22<02:37,  2.86s/it][A

100%|██████████| 1/1 [00:03<00:00,  3.64s/it]


  6%|▌         | 3/51 [00:08<02:25,  3.04s/it]

  8%|▊         | 4/51 [00:11<02:11,  2.81s/it]

 35%|███▌      | 28/80 [01:29<02:11,  2.53s/it][A


 10%|▉         | 5/51 [00:13<02:06,  2.74s/it]

 12%|█▏        | 6/51 [00:16<01:59,  2.67s/it]


 38%|███▊      | 30/80 [01:33<01:59,  2.39s/it][A

 14%|█▎        | 7/51 [00:19<01:59,  2.71s/it]


 20%|██        | 2/10 [00:06<00:26,  3.28s/it]

 39%|███▉    

 98%|█████████▊| 50/51 [02:13<00:02,  2.83s/it]

 92%|█████████▎| 74/80 [03:31<00:17,  2.88s/it][A


100%|██████████| 51/51 [02:16<00:00,  2.67s/it]


 94%|█████████▍| 75/80 [03:34<00:14,  2.83s/it][A


 44%|████▍     | 32/72 [01:27<01:50,  2.76s/it]

 95%|█████████▌| 76/80 [03:37<00:11,  2.93s/it][A


 46%|████▌     | 33/72 [01:30<01:49,  2.80s/it]

 96%|█████████▋| 77/80 [03:40<00:09,  3.02s/it]


 47%|████▋     | 34/72 [01:33<01:48,  2.86s/it]

 98%|█████████▊| 78/80 [03:43<00:05,  2.86s/it]


 49%|████▊     | 35/72 [01:35<01:38,  2.68s/it]

 75%|███████▌  | 12/16 [00:35<00:11,  2.82s/it]


100%|██████████| 80/80 [03:47<00:00,  2.85s/it]


 81%|████████▏ | 13/16 [00:38<00:08,  2.81s/it]


  0%|          | 0/84 [00:00<?, ?it/s]2.59s/it]

 88%|████████▊ | 14/16 [00:41<00:05,  2.82s/it]


 53%|█████▎    | 38/72 [01:43<01:29,  2.65s/it]

  1%|          | 1/84 [00:04<05:34,  4.04s/it]]


  2%|▏         | 2/84 [00:10<06:24,  4.69s/it]

100%|██████████| 16/16 [00:50<00:00,  3.99s/it]


100

 32%|███▏      | 38/119 [02:16<03:35,  2.66s/it]


 20%|██        | 13/64 [00:41<02:23,  2.81s/it]

 33%|███▎      | 39/119 [02:18<03:17,  2.47s/it]


 22%|██▏       | 14/64 [00:43<02:14,  2.68s/it]

 34%|███▎      | 40/119 [02:21<03:17,  2.51s/it]


 23%|██▎       | 15/64 [00:45<02:08,  2.62s/it]

 34%|███▍      | 41/119 [02:24<03:26,  2.65s/it]


 25%|██▌       | 16/64 [00:48<02:04,  2.58s/it]

 35%|███▌      | 42/119 [02:26<03:17,  2.57s/it]


 27%|██▋       | 17/64 [00:51<02:01,  2.58s/it]

 36%|███▌      | 43/119 [02:28<03:14,  2.56s/it]


 28%|██▊       | 18/64 [00:53<02:01,  2.64s/it]

 37%|███▋      | 44/119 [02:35<04:45,  3.81s/it]

 57%|█████▋    | 20/35 [01:03<00:54,  3.66s/it]


 38%|███▊      | 45/119 [02:39<04:49,  3.92s/it]

 67%|██████▋   | 56/84 [03:35<01:36,  3.43s/it]


 39%|███▊      | 46/119 [02:42<04:19,  3.55s/it]

 68%|██████▊   | 57/84 [03:37<01:25,  3.17s/it]


 39%|███▉      | 47/119 [02:45<04:05,  3.40s/it]


 34%|███▍      | 22/64 [01:11<02:32,  3.62s/it]



...

 91%|█████████ | 105/116 [07:37<00:37,  3.45s/it]


 33%|███▎      | 29/87 [01:29<02:46,  2.86s/it]

 74%|███████▍  | 74/100 [04:06<01:16,  2.95s/it]


 34%|███▍      | 30/87 [01:32<02:49,  2.97s/it]t]

 92%|█████████▏| 107/116 [07:43<00:29,  3.30s/it]


 36%|███▌      | 31/87 [01:35<02:38,  2.83s/it]


 93%|█████████▎| 108/116 [07:46<00:25,  3.17s/it]

 94%|█████████▍| 109/116 [07:49<00:20,  2.93s/it]


 38%|███▊      | 33/87 [01:41<02:37,  2.92s/it]

 95%|█████████▍| 110/116 [07:51<00:16,  2.83s/it]


 39%|███▉      | 34/87 [01:43<02:32,  2.88s/it]

 96%|█████████▌| 111/116 [07:55<00:15,  3.04s/it]


 40%|████      | 35/87 [01:47<02:45,  3.18s/it]

 79%|███████▉  | 79/100 [04:23<01:11,  3.39s/it]


100%|██████████| 68/68 [03:53<00:00,  3.44s/it]

 41%|████▏     | 36/87 [01:50<02:38,  3.11s/it]t]

 43%|████▎     | 37/87 [01:54<02:46,  3.32s/it]t]

 98%|█████████▊| 114/116 [08:04<00:06,  3.08s/it]

 99%|█████████▉| 115/116 [08:10<00:03,  3.88s/it]

 83%|████████▎ | 83/100 [04:38<01:03,

 21%|██        | 8/38 [00:26<01:24,  2.81s/it]


 24%|██▎       | 9/38 [00:28<01:16,  2.64s/it]


 13%|█▎        | 19/146 [01:03<08:24,  3.97s/it]

 26%|██▋       | 10/38 [00:30<01:10,  2.53s/it]


 14%|█▎        | 20/146 [01:06<07:41,  3.66s/it]

 29%|██▉       | 11/38 [00:33<01:10,  2.61s/it]


 14%|█▍        | 21/146 [01:09<06:41,  3.21s/it]

  5%|▍         | 7/143 [00:25<08:12,  3.62s/it][A


 32%|███▏      | 12/38 [00:36<01:11,  2.76s/it]

  6%|▌         | 8/143 [00:28<07:25,  3.30s/it][A


 34%|███▍      | 13/38 [00:39<01:08,  2.74s/it]

  6%|▋         | 9/143 [00:30<06:45,  3.02s/it][A


 16%|█▋        | 24/146 [01:16<05:31,  2.72s/it]

 37%|███▋      | 14/38 [00:43<01:12,  3.01s/it]


 17%|█▋        | 25/146 [01:18<05:08,  2.55s/it]

  8%|▊         | 11/143 [00:36<06:43,  3.05s/it]A


 39%|███▉      | 15/38 [00:46<01:13,  3.18s/it]

 80%|████████  | 8/10 [00:22<00:05,  2.94s/it]


 42%|████▏     | 16/38 [00:49<01:08,  3.10s/it]

 90%|█████████ | 9/10 [00:25<00:02,  2.81s/it]




 41%|████▏     | 59/143 [03:34<02:42,  1.93s/it]

 10%|▉         | 7/71 [00:14<02:11,  2.05s/it]


 42%|████▏     | 60/143 [03:37<03:00,  2.17s/it]

 17%|█▋        | 8/46 [00:19<01:48,  2.86s/it]


 43%|████▎     | 61/143 [03:39<02:56,  2.15s/it]

 13%|█▎        | 9/71 [00:18<02:14,  2.17s/it]


 43%|████▎     | 62/143 [03:42<03:06,  2.31s/it]A


 22%|██▏       | 10/46 [00:23<01:27,  2.44s/it]

 44%|████▍     | 63/143 [03:44<02:59,  2.24s/it][A


 53%|█████▎    | 78/146 [04:29<02:17,  2.02s/it]

 45%|████▍     | 64/143 [03:45<02:40,  2.04s/it][A


 26%|██▌       | 12/46 [00:27<01:14,  2.19s/it]

 45%|████▌     | 65/143 [03:48<02:43,  2.09s/it][A

 28%|██▊       | 13/46 [00:29<01:10,  2.14s/it]


 46%|████▌     | 66/143 [03:50<02:42,  2.11s/it]


 55%|█████▌    | 81/146 [04:35<02:08,  1.98s/it]

 47%|████▋     | 67/143 [03:52<02:36,  2.05s/it][A


 33%|███▎      | 15/46 [00:33<01:04,  2.09s/it]

 48%|████▊     | 68/143 [03:54<02:36,  2.08s/it][A


 35%|███▍      | 16/46 [00:35<01:03,  2

 75%|███████▍  | 107/143 [05:56<02:21,  3.94s/it]A


 41%|████      | 7/17 [00:20<00:28,  2.80s/it]

 76%|███████▌  | 108/143 [05:59<02:07,  3.64s/it]A


 76%|███████▌  | 109/143 [06:02<01:52,  3.30s/it]


 87%|████████▋ | 127/146 [06:46<00:54,  2.87s/it]

 77%|███████▋  | 110/143 [06:04<01:38,  2.99s/it]


 88%|████████▊ | 128/146 [06:49<00:51,  2.88s/it]

 78%|███████▊  | 111/143 [06:08<01:47,  3.36s/it]A


 88%|████████▊ | 129/146 [06:53<00:52,  3.07s/it]

 78%|███████▊  | 112/143 [06:11<01:41,  3.26s/it]A


 89%|████████▉ | 130/146 [06:56<00:50,  3.14s/it]

 71%|███████   | 12/17 [00:35<00:14,  2.89s/it]


 90%|████████▉ | 131/146 [06:59<00:44,  2.98s/it]

 76%|███████▋  | 13/17 [00:38<00:11,  2.98s/it]


 90%|█████████ | 132/146 [07:02<00:41,  2.99s/it]

 82%|████████▏ | 14/17 [00:41<00:08,  2.82s/it]

 90%|█████████ | 64/71 [02:57<00:19,  2.84s/it]


 88%|████████▊ | 15/17 [00:43<00:05,  2.71s/it]


 92%|█████████▏| 134/146 [07:07<00:34,  2.90s/it]

 81%|████████  | 116/143 [06:2

 71%|███████   | 32/45 [01:31<00:31,  2.39s/it]

 43%|████▎     | 9/21 [00:29<00:33,  2.81s/it]]


 73%|███████▎  | 33/45 [01:33<00:28,  2.36s/it]

 48%|████▊     | 10/21 [00:31<00:28,  2.56s/it]


 76%|███████▌  | 34/45 [01:35<00:26,  2.37s/it]

 52%|█████▏    | 11/21 [00:33<00:23,  2.39s/it]


 78%|███████▊  | 35/45 [01:37<00:21,  2.17s/it]

 57%|█████▋    | 12/21 [00:35<00:19,  2.14s/it]

 57%|█████▋    | 29/51 [01:15<00:43,  1.99s/it]


 62%|██████▏   | 13/21 [00:38<00:18,  2.34s/it]

 59%|█████▉    | 30/51 [01:17<00:43,  2.06s/it]


 67%|██████▋   | 14/21 [00:39<00:14,  2.11s/it]

 61%|██████    | 31/51 [01:19<00:38,  1.91s/it]


 71%|███████▏  | 15/21 [00:41<00:12,  2.03s/it]

 63%|██████▎   | 32/51 [01:22<00:40,  2.13s/it]


 76%|███████▌  | 16/21 [00:43<00:10,  2.14s/it]

 65%|██████▍   | 33/51 [01:23<00:36,  2.05s/it]


 81%|████████  | 17/21 [00:45<00:08,  2.06s/it]

 67%|██████▋   | 34/51 [01:25<00:32,  1.90s/it]


 86%|████████▌ | 18/21 [00:47<00:05,  1.96s/it]

 69%|██████

In [5]:
driver.close()

In [ ]:
### 이거능 ... 업종 넘버 지정해서 멀티스레딩 하기 


# # 업종별 페이지 url 넘버와 저장할 csv파일 이름을 지정합니당 
# food = [38,'식품전체']
# heldevice=[211,'건강관리장비와용품전체']
# elecdevice=[221,'전기제품전체']
# metal = [62,'철강전체']

# # 한번 더 묶어줍니당 
# list_ups = [food,heldevice,elecdevice,metal]
